In [69]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from App.helper.functions import station_upper
import App.helper.config as config

In [70]:
# Read twitter data
# daten = pd.read_csv('App/Daten/data_expanded/combined_data_twitter_long.csv')
daten1 = pd.read_csv('App/Daten/data_new/combined_data_twitter_long_1.csv')
daten2 = pd.read_csv('App/Daten/data_new/combined_data_twitter_long_2.csv')
daten = pd.concat([daten1, daten2], ignore_index=True, axis=0)
daten['Einschr_type'].fillna('Keine', inplace=True)
print(len(daten))
daten.head(2)

988164


,BETRIEBSTAG,LINIEN_ID,LINIEN_TEXT,haltestelle_an,AN_soll,AN_ist,AN_diff,haltestelle_ab,AB_soll,AB_ist,...,weekday,ab_hour,ab_minute,disturbance_overlap,Einschr_type,feiertag,Temperatur,Niederschlag,Luftfeuchtigkeit,Wind
0,2022-05-24,2021,IR,St. Gallen,2022-05-24 12:55:00,2022-05-24 12:55:40,0.666667,Luzern,2022-05-24 10:39:00,2022-05-24 10:39:34,...,1,10,39,False,Keine,0,12.9,8.1,88.4,4.0
1,2023-04-28,505,IC5,St. Gallen,2023-04-28 07:02:00,2023-04-28 07:02:40,0.666667,Zürich HB,2023-04-28 06:03:00,2023-04-28 06:03:17,...,4,6,3,False,Keine,0,14.1,22.5,93.3,3.6


In [71]:
daten.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 988164 entries, 0 to 988163
Data columns (total 21 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   BETRIEBSTAG          988164 non-null  object 
 1   LINIEN_ID            988164 non-null  int64  
 2   LINIEN_TEXT          988164 non-null  object 
 3   haltestelle_an       988164 non-null  object 
 4   AN_soll              988164 non-null  object 
 5   AN_ist               988164 non-null  object 
 6   AN_diff              988164 non-null  float64
 7   haltestelle_ab       988164 non-null  object 
 8   AB_soll              988164 non-null  object 
 9   AB_ist               988164 non-null  object 
 10  AB_diff              988164 non-null  float64
 11  weekday              988164 non-null  int64  
 12  ab_hour              988164 non-null  int64  
 13  ab_minute            988164 non-null  int64  
 14  disturbance_overlap  988164 non-null  bool   
 15  Einschr_type     

In [72]:
daten['Linien_typ'] = daten['LINIEN_TEXT'].str.replace('\d+', '', regex=True)

In [73]:
daten['Linien_typ'].value_counts()

IR     471045
IC     430970
EC      61154
ICE     24995
Name: Linien_typ, dtype: int64

In [74]:
daten.isna().sum().sort_values(ascending=False).head(1)

Wind    454057
dtype: int64

In [75]:
# replace True and False with 1 and 0 in the column 'disturbance_overlap'
daten['disturbance_overlap'] = daten['disturbance_overlap'].replace(True, 1)
daten['disturbance_overlap'] = daten['disturbance_overlap'].replace(False, 0)

In [76]:
daten.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 988164 entries, 0 to 988163
Data columns (total 22 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   BETRIEBSTAG          988164 non-null  object 
 1   LINIEN_ID            988164 non-null  int64  
 2   LINIEN_TEXT          988164 non-null  object 
 3   haltestelle_an       988164 non-null  object 
 4   AN_soll              988164 non-null  object 
 5   AN_ist               988164 non-null  object 
 6   AN_diff              988164 non-null  float64
 7   haltestelle_ab       988164 non-null  object 
 8   AB_soll              988164 non-null  object 
 9   AB_ist               988164 non-null  object 
 10  AB_diff              988164 non-null  float64
 11  weekday              988164 non-null  int64  
 12  ab_hour              988164 non-null  int64  
 13  ab_minute            988164 non-null  int64  
 14  disturbance_overlap  988164 non-null  int64  
 15  Einschr_type     

In [77]:
data = daten.drop(['AN_soll', 'AN_ist', 'AB_soll', 'AB_ist', 'AB_diff', 'LINIEN_ID', 'LINIEN_TEXT', 'disturbance_overlap'], axis=1)
data = pd.get_dummies(data, columns=['haltestelle_ab', 'haltestelle_an', 'Linien_typ', 'Einschr_type'])

In [78]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 988164 entries, 0 to 988163
Data columns (total 33 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   BETRIEBSTAG                  988164 non-null  object 
 1   AN_diff                      988164 non-null  float64
 2   weekday                      988164 non-null  int64  
 3   ab_hour                      988164 non-null  int64  
 4   ab_minute                    988164 non-null  int64  
 5   feiertag                     988164 non-null  int64  
 6   Temperatur                   534107 non-null  float64
 7   Niederschlag                 534107 non-null  float64
 8   Luftfeuchtigkeit             534107 non-null  float64
 9   Wind                         534107 non-null  float64
 10  haltestelle_ab_Basel SBB     988164 non-null  uint8  
 11  haltestelle_ab_Bern          988164 non-null  uint8  
 12  haltestelle_ab_Lugano        988164 non-null  uint8  
 13 

In [79]:
data1 = data.loc[len(data)//2:]
data2 = data.loc[:len(data)//2]
data1.to_csv('App/Daten/data_new/less_wide1.csv', index=False)
data2.to_csv('App/Daten/data_new/less_wide2.csv', index=False)

In [80]:
data_detail = daten.drop(['AN_soll', 'AN_ist', 'AB_soll', 'AB_ist', 'AB_diff', 'LINIEN_ID', 'Linien_typ', 'disturbance_overlap'], axis=1)
data_detail['week'] = pd.to_datetime(data_detail['BETRIEBSTAG']).dt.isocalendar().week
data_detail = pd.get_dummies(data_detail, columns=['haltestelle_ab', 'haltestelle_an', 'LINIEN_TEXT', 'Einschr_type'])

In [81]:
data_detail1 = data_detail.loc[len(data_detail)//2:]
data_detail2 = data_detail.loc[:len(data_detail)//2]
data_detail1.to_csv('App/Daten/data_new/detail1.csv', index=False)
data_detail2.to_csv('App/Daten/data_new/detail2.csv', index=False)